# Introduction of LangChain

In [1]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os

# Set the API key
os.environ['OPENAI_API_KEY'] = "xxx"

# Now you can use the API key
openai_api_key = os.getenv('OPENAI_API_KEY')

## Chat Models: GPT-3.5 Turbo and GPT-4

In [32]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()  

# invoking the llm (running the prompt)
output = llm.invoke('Explain Canada in one sentence.', model='gpt-4-turbo', temperature=0.1)
print(output.content)

Canada is a vast, multicultural country in North America known for its natural beauty, politeness, and strong commitment to human rights and multiculturalism.


In [36]:
# using Chat Completions API Messages: System, Assistant and Human
from langchain.schema import(
    SystemMessage, 
    AIMessage,
    HumanMessage
)
messages = [
    SystemMessage(content='You are a physicist and respond only in Chinese.'),
    HumanMessage(content='Explain Canada in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

加拿大是一个北美洲的国家，拥有广阔的土地、丰富的资源和多元的文化。


## Caching LLM Responses

In [39]:
# in-memory cache
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [41]:
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

In [43]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: user 15.8 ms, sys: 2.64 ms, total: 18.4 ms
Wall time: 615 ms


"\n\nWhy did the banana go to the doctor?\n\nBecause it wasn't peeling well!"

In [45]:
%%time
llm.invoke(prompt)

CPU times: user 429 μs, sys: 61 μs, total: 490 μs
Wall time: 554 μs


"\n\nWhy did the banana go to the doctor?\n\nBecause it wasn't peeling well!"

In [51]:
#sqlite cache
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


In [53]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

CPU times: user 19.9 ms, sys: 2.99 ms, total: 22.9 ms
Wall time: 622 ms


"\nWhy don't scientists trust atoms?\n\nBecause they make up everything. "

In [55]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

CPU times: user 8.08 ms, sys: 5.13 ms, total: 13.2 ms
Wall time: 12.5 ms


"\nWhy don't scientists trust atoms?\n\nBecause they make up everything. "

## LLM Streaming

In [58]:
# the default mode is llm will return the complete task then return

from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

Verse 1:
In the dead of night, under the glow
Of the silvery moon, a raven flies low
Its wings beat against the midnight sky
As it calls out a haunting cry

Chorus:
Oh, moon and raven, dance in the night
Their mysterious ways, hidden from sight
Together they roam, in the darkness they thrive
A bond unbreakable, they'll never divide

Verse 2:
The moon whispers secrets to the raven's ear
As they soar through the heavens, without fear
They're creatures of the night, wild and free
A sight to behold, for all to see

Chorus:
Oh, moon and raven, dance in the night
Their mysterious ways, hidden from sight
Together they roam, in the darkness they thrive
A bond unbreakable, they'll never divide

Bridge:
They're symbols of mystery, magic, and lore
In their presence, you can't help but adore
The moon and the raven, a sight so divine
Their beauty and power, forever intertwine

Chorus:
Oh, moon and raven, dance in the night
Their mysterious ways, hidden from sight
Together they roam, in the darkness

In [60]:
# enable streaming
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Verse 1:
The moon hangs high in the midnight sky
Casting shadows on the land below
A raven calls out in the dead of night
Its cry echoing through the darkened world

Chorus:
Moon and raven, spirits of the night
Dancing in the shadows, bathed in silver light
Mysteries unfolding in the pale moon's glow
Moon and raven, forever intertwined

Verse 2:
The raven perches on a gnarled old tree
Its eyes gleaming with ancient wisdom
It watches as the moon rises higher
A silent sentinel in the velvet sky

Chorus:
Moon and raven, spirits of the night
Dancing in the shadows, bathed in silver light
Mysteries unfolding in the pale moon's glow
Moon and raven, forever intertwined

Bridge:
They are the guardians of the night
A pair of souls forever bound
In the darkness they find their light
In the silence, their secrets are found

Chorus:
Moon and raven, spirits of the night
Dancing in the shadows, bathed in silver light
Mysteries unfolding in the pale moon's glow
Moon and raven, forever intertwined

Ou

## PromptTemplates

In [69]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a template for the prompt
template = '''
You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}.
'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template=template)

# Fill in the variable: virus and language
prompt = prompt_template.format(virus='zita', language='Chinese')
prompt  # Returns the generated prompt


'\nYou are an experienced virologist.\nWrite a few sentences about the following virus "zita" in Chinese.\n'

In [71]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

"Zita" 病毒是一种新型冠状病毒，最初在动物中发现，后来传播给人类。这种病毒具有高度传染性，可引起呼吸道疾病和肺部感染。目前尚无特效药物治疗，预防措施包括保持良好的个人卫生和避免密切接触感染者。


## ChatPromptTemplates

In [74]:
# chat promopt tenplates are for conversations

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')
print(messages)  # Outputs the formatted chat messages


[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 5 countries in World by population.')]


In [76]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "countries": [
        {
            "name": "China",
            "population": "1,439,323,776"
        },
        {
            "name": "India",
            "population": "1,380,004,385"
        },
        {
            "name": "United States",
            "population": "331,002,651"
        },
        {
            "name": "Indonesia",
            "population": "273,523,615"
        },
        {
            "name": "Pakistan",
            "population": "220,892,340"
        }
    ]
}


## Simple Chains

In [85]:
# for complex conversations, we need chains to combine steps/components 

from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
# import llmchain
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

# create a simple chain
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'zita', 'language': 'Chinese'})




> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist.
Write a few sentences about the following virus "zita" in Chinese.

> Finished chain.


In [86]:
print(output)

{'virus': 'zita', 'language': 'Chinese', 'text': '"Zita"病毒是一种新型冠状病毒，最初在中国武汉市被发现。这种病毒能够引起呼吸道感染，导致发热、咳嗽和呼吸困难等症状。目前全球范围内都在努力应对这种病毒的传播。'}


In [89]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

# Initialize an LLMChain with the ChatOpenAI model and the prompt template
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

# enter a counrty first
country = input('Enter Country: ')

# Invoke the chain with specific virus and language values
output = chain.invoke(country)
print(output['text'])

Enter Country:  China




> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of China?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of China is Beijing.

- The Great Wall of China
- The Forbidden City
- The Temple of Heaven


## Sequential Chains

In [92]:

# for more complex task
# SimpleSequentialChain - answer as next chain's input
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-3.5-turbo) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4-turbo) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')




> Entering new SimpleSequentialChain chain...
Sure! Here is a simple implementation of linear regression in Python:

```python
import numpy as np

def linear_regression(X, y):
    X = np.array(X)
    y = np.array(y)
    
    n = X.shape[0]
    m = X.shape[1]
    
    X_b = np.c_[np.ones((n, 1)), X]  # add bias term
    
    theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)  # calculate theta
    
    return theta

# Example usage
X = np.array([[1, 2], [2, 3], [3, 4]])
y = np.array([2, 3, 4])

theta = linear_regression(X, y)
print(theta)
```

This function takes in a matrix X (features) and a vector y (target values) and calculates the optimal parameters theta using the normal equation. The function returns the parameters theta that minimize the mean squared error between the predicted values and the actual values.
The Python function `linear_regression` demonstrates a fundamental algorithm in machine learning and statistics: Linear Regression. This algorithm aims to find a line

# LangChain Agents in Action: Python REPL

In [97]:
pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [99]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [101]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model with gpt-4-turbo and a temperature of 0
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

# Create a Python agent using the ChatOpenAI model and a PythonREPLTool
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# Invoke the agent
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To solve this, I need to calculate the factorial of 12 first, then find its square root, and finally format the result to display it with 4 decimal points. I can use the `math` module in Python to accomplish this, as it provides both `factorial` and `sqrt` functions.

Action: Python_REPL
Action Input: import math
Observation: 
Thought:Now that the `math` module is imported, I can use it to calculate the factorial of 12.

Action: Python_REPL
Action Input: factorial_12 = math.factorial(12)
Observation: 
Thought:With the factorial of 12 calculated, I can now proceed to find its square root.

Action: Python_REPL
Action Input: sqrt_factorial_12 = math.sqrt(factorial_12)
Observation: 
Thought:Now that I have the square root of the factorial of 12, I need to format it to display with 4 decimal points.

Action: Python_REPL
Action Input: print(f"{sqrt_factorial_12:.4f}")
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [103]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [105]:
response

{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}

## LangChain Tools: DuckDuckGo and Wikipedia

In [111]:
# duckduckgo
#!pip install -q duckduckgo-search

In [115]:
# optional, filter workings (recommended in production)
import warnings
warnings.filterwarnings('ignore', module='langchain')

In [135]:
#  DuckDuckGo search component.
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.invoke('Which counrty has the most number of gold medal in 2024 Olympics in Paries')
print(output)

It's the final day of the Summer Olympics and the USA and China have tied in the overall gold medal rankings from these 2024 Summer Olympics. While Team USA is far ahead in the total medal ... A complete look at which countries are leading the medal count at the 2024 Paris Olympics. With the Paris Olympics complete, the U.S. topped the total medal count and tied China for the most golds. China held a slight lead over the U.S. entering Sunday and claimed its final gold medal ... The contest for most gold medals at the Paris Olympics ended in a 40-40 tie between China and the United States. With the 2024 Paris Olympics about to close, the United States will end the Summer Games as the country that has won more medals than any other nation.


In [136]:
search.name

'duckduckgo_search'

In [137]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [138]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
output = search.run('Which counrty has the most number of gold medal in 2024 Olympics in Paries')
print(output)

[snippet: It's the final day of the Summer Olympics and the USA and China have tied in the overall gold medal rankings from these 2024 Summer Olympics. While Team USA is far ahead in the total medal ..., title: USA and China tie for most gold medals in the 2024 Summer Olympics - CNN, link: https://www.cnn.com/2024/08/11/sport/olympics-medal-rankings-spt-intl/index.html], [snippet: The U.S. won one more gold than it did in the Tokyo Games three years ago, and America has gone plus-three in its silver tally, up from 41, and far surpassed the previous number of bronze medals ..., title: 2024 Olympics medal count: United States wins race, ties China with 40 ..., link: https://www.cbssports.com/olympics/news/2024-paris-olympics-medal-count-tracker-for-how-many-gold-silver-bronze-medals-usa-each-country-has-won/], [snippet: The United States again comfortably finished with the most medals at the 2020 Olympics in Tokyo, winning 113 overall and 39 gold, but the gap narrowed from 2016 when the 

In [145]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region='de-de', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('2024 Paries Olympics')

In [147]:
print(output)

[snippet: Record-breaking US 4x400m quartets bring Paris 2024 Olympics track and field programme to a thrilling finale. Ethiopia's late sub Tola wins marathon in Olympic record in brutal conditions over hugely challenging course. Paulino and Benjamin dazzle over one lap to win Olympic golds in Paris., title: Home | Paris 24 | Olympic Games - World Athletics, link: https://worldathletics.org/competitions/olympic-games/paris24], [snippet: Driven by the reforms of Olympic Agenda 2020, Paris 2024 set a precedent for future editions. Find out here how the IOC supported the Organising Committee and get access to key documents like the Host City Contract, qualification information, anti-doping and medical rules as well as media guidelines and News Access Rules., title: IOC Paris 2024 Gallery - Olympics.com, link: https://olympics.com/ioc/paris-2024], [snippet: Check out the full competition schedule for Paris 2024 on Olympics.com, with information on when every event will take place during th

In [149]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

Snippet: Record-breaking US 4x400m quartets bring Paris 2024 Olympics track and field programme to a thrilling finale. Ethiopia's late sub Tola wins marathon in Olympic record in brutal conditions over hugely challenging course. Paulino and Benjamin dazzle over one lap to win Olympic golds in Paris.
Title: Home | Paris 24 | Olympic Games - World Athletics
Link: https://worldathletics.org/competitions/olympic-games/paris24

--------------------------------------------------
Snippet: Driven by the reforms of Olympic Agenda 2020, Paris 2024 set a precedent for future editions. Find out here how the IOC supported the Organising Committee and get access to key documents like the Host City Contract, qualification information, anti-doping and medical rules as well as media guidelines and News Access Rules.
Title: IOC Paris 2024 Gallery - Olympics.com
Link: https://olympics.com/ioc/paris-2024

--------------------------------------------------
Snippet: Check out the full competition schedule f

## Wikipedia 

In [152]:
!pip install -q wikipedia

In [153]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [158]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=10000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'Llama3'})

'Page: Llama (language model)\nSummary: Llama (acronym for Large Language Model Meta AI, and formerly stylized as LLaMA) is a family of autoregressive large language models (LLMs) released by Meta AI starting in February 2023. The latest version is Llama 3.1, released in July 2024.\nModel weights for the first version of Llama were made available to the research community under a non-commercial license, and access was granted on a case-by-case basis. Unauthorized copies of the model were shared via BitTorrent. In response, Meta AI issued DMCA takedown requests against repositories sharing the link on GitHub. Subsequent versions of Llama were made accessible outside academia and released under licenses that permitted some commercial use. Llama models are trained at different parameter sizes, ranging between 7B and 405B. Originally, Llama was only available as a foundation model. Starting with Llama 2, Meta AI started releasing instruction fine-tuned versions alongside foundation models.

In [170]:
output = wiki.invoke("Toronto")

In [171]:
print(output)

Page: Toronto
Summary: Toronto is the most populous city in Canada and the capital city of the Canadian province of Ontario. With a population of 2,794,356 in 2021, it is the fourth-most populous city in North America. The city is the anchor of the Golden Horseshoe, an urban agglomeration of 9,765,188 people (as of 2021) surrounding the western end of Lake Ontario, while the Greater Toronto Area proper had a 2021 population of 6,712,341. Toronto is an international centre of business, finance, arts, sports, and culture and is one of the most multicultural and cosmopolitan cities in the world.
Indigenous peoples have travelled through and inhabited the Toronto area, located on a broad sloping plateau interspersed with rivers, deep ravines, and urban forest, for more than 10,000 years. After the broadly disputed Toronto Purchase, when the Mississauga surrendered the area to the British Crown, the British established the town of York in 1793 and later designated it as the capital of Upper

## Creating a ReAct Agent

In [ ]:
# reasoning and acting (ReAct)
# agents = tools + chains

In [175]:
pip install langchainhub -q

Note: you may need to restart the kernel to use updated packages.


In [191]:
LANGCHAIN_HUB_API_KEY="xxx"

In [195]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model (gpt-4-turbo-preview) with a temperature of 0. Utilize gpt-3.5-turbo if you use the free plan
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

# Define a template for answering questions
template = '''
Answer the following questions in Chinese as best you can.
Questions: {q}
'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template)

# Pull the react prompt from the hub
prompt = hub.pull('hwchase17/react', api_key=LANGCHAIN_HUB_API_KEY)

# displaying information about the react prompt
print(type(prompt))
print(prompt.input_variables)
print(prompt.template)



<class 'langchain_core.prompts.prompt.PromptTemplate'>
['agent_scratchpad', 'input', 'tool_names', 'tools']
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [197]:

# Create three tools for the agent

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

# Combine the tools into a list
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Create a react agent with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)


In [199]:
# question = 'Generate the first 20 numbers in the Fibonacci series.'
# question = 'Who is the current prime minister of the Canada?'
question = 'Tell me about Napoleon Bonaparte early life'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})




> Entering new AgentExecutor chain...
I should use Wikipedia to look up information about Napoleon Bonaparte's early life in Chinese.

Action: Wikipedia
Action Input: "拿破仑·波拿巴早年生活"No good Wikipedia Search Result was foundI should try a DuckDuckGo search instead to find information about Napoleon Bonaparte's early life in Chinese.

Action: DuckDuckGo Search
Action Input: "拿破仑·波拿巴早年生活"探索位于地中海科西嘉岛的拿破仑·波拿巴的出生地阿雅克肖，以及他在法国历史上留下的遗产。 ... 拿破仑在阿雅克肖的早年生活. 公共领域，通过Wikimedia Commons. 1769年，拿破仑出生于法国科西嘉岛的阿雅克修镇。据报道，就在前一年，法国从意大利城邦热那亚 ... 波拿巴王朝（ Bonaparte ，1804-1814年，1815年，1852-1870年）是欧洲 法国一个王朝，1804年由第一执政拿破仑·波拿巴建立。. 拿破仑一世在1799年11月9日发动军事政变取得政权，结束法国大革命。 他其后经法国人民投票先后成为终身执政及皇帝，把法兰西第一共和国改制成法兰西第一帝国，其后经过一连串 ... 拿破崙三世（法語： Napoléon III ，1808年4月20日—1873年1月9日），即夏尔·路易·拿破仑·波拿巴（法語： Charles Louis Napoléon Bonaparte ），法兰西第二共和国唯一一位总统及法兰西第二帝国唯一一位皇帝，亦是拿破仑一世的侄子和继承人，亦是拿破仑一世的皇后約瑟 ... 一、早年生活. 拿破仑·波拿巴于1769年8月15日出生在法国科西嘉岛的一个小村庄，他是家中的第二个儿子。尽管他的家庭并不富裕，但拿破仑的父母都受过良好的教育，注重子女的教育。这种家庭背景让拿破仑在早年就展现出卓越的领导才能和野心。 波拿巴家族于17 世纪末搬进了这座房子的一部分，从